In [239]:
import requests
import nltk
import string
import gensim
import pandas as pd
from nltk.corpus import stopwords, wordnet
from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.probability import FreqDist
from googletrans import Translator
from bs4 import BeautifulSoup
from gensim import corpora, models

In [241]:
soup = BeautifulSoup(page.content, 'html.parser')

In [242]:
for i in soup:
    print(i)

html


<html lang="es">
<head>
<meta charset="utf-8"/>
<link href="https://www.googletagmanager.com" rel="preconnect"/>
<link href="https://tpc.googlesyndication.com" rel="dns-prefetch"/>
<link href="https://securepubads.g.doubleclick.net" rel="dns-prefetch"/>
<link href="https://pagead2.googlesyndication.com" rel="preconnect"/>
<link href="https://googleads.g.doubleclick.net" rel="preconnect"/>
<link href="https://securepubads.g.doubleclick.net" rel="preconnect"/>
<link href="https://i.musicaimg.com" rel="preconnect"/>
<link href="https://www.google-analytics.com" rel="dns-prefetch"/>
<script>
function adsss(useGooglePersonalization){
  //analytics
  let script_a = document.createElement('script');
  script_a.src = "https://www.googletagmanager.com/gtag/js?id=UA-5877893-4";
  script_a.async = true;
  document.body.append(script_a);
  window.dataLayer = window.dataLayer || [];
  function gtag(){dataLayer.push(arguments);}
  gtag('js', new Date());
  if(useGooglePersonalization) {gtag('

In [243]:
p_l = []
for p in soup.find_all('p'):
    p_l.append(p)
    
raw_song = list(p_l[1])
filtered_song = []
for j in raw_song:
    if ('p***' in j):
        filtered_song.append(str(j).replace('p***', 'puta'))
        if ('mie***' in j):
            filtered_song.append(str(j).replace('mie***', 'mierda'))
        else: continue
    elif ('mie***' in j):
        filtered_song.append(str(j).replace('mie***', 'mierda'))
        if ('p***' in j):
            filtered_song.append(str(j).replace('p***', 'puta'))
        else: continue
    else:
        filtered_song.append(str(j))  
    
filtered_song = filtered_song[::2]

In [244]:
filtered_song[0:5]

['    Mi padre es el sol, mi madre la luna',
 ' mi hermano es el viento y el planeta tierra mi cuna',
 ' mis unicos hijos son las frases que me invento',
 ' y mi mayor regalo es vivir este momento',
 ' en el que siento que callar es un pecado capital']

In [245]:
translator = Translator()
translate_song = []
for i in translator.translate(filtered_song):
    translate_song.append(i.text)

In [246]:
translate_song[0:5]

['My father is the sun, the moon my mother',
 'my brother is the wind and planet earth my crib',
 'my only children are the phrases that invented me',
 'and my greatest gift is to live this time',
 'in which I feel that silence is a cardinal sin']

In [247]:
word_tokenize = word_tokenize(str(translate_song).lower())

In [248]:
stop_words = stopwords.words('english')
filtered_words = [x for x in word_tokenize if x not in stop_words +list(string.punctuation)+['``',"''"]]

In [249]:
for i, word in enumerate(filtered_words):
    for char in word:
        if char == "'":
            word = word.replace("'",'')
            filtered_words[i] = word
        else:
            filtered_words[i] = word

In [252]:
FreqDist([x for x in filtered_words if not x in stop_words]).most_common(20)

[('rap', 10),
 ('children', 6),
 ('life', 6),
 ('know', 5),
 ('nothing', 5),
 ('see', 5),
 ('go', 5),
 ('want', 4),
 ('another', 4),
 ('look', 4),
 ('like', 4),
 ('also', 4),
 ('give', 4),
 ('although', 4),
 ('love', 4),
 ('time', 3),
 ('feel', 3),
 ('city', 3),
 ('science', 3),
 ('tell', 3)]

In [253]:
translate_song = pd.DataFrame({'Sentences': translate_song})

In [254]:
def get_wordnet_pos(word):
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}
    
    return tag_dict.get(tag, wordnet.NOUN)

In [255]:
def preprocess(translate_song):
    lem = WordNetLemmatizer()
    results = []
    for token in gensim.utils.simple_preprocess(translate_song):
        if token not in list(gensim.parsing.preprocessing.STOPWORDS) +list(string.punctuation):
            results.append(lem.lemmatize(token, get_wordnet_pos(token)))      
    
    return results

In [256]:
preprocess_sentences = translate_song['Sentences'].map(preprocess)

In [257]:
preprocess_sentences[0:5]

0             [father, sun, moon, mother]
1    [brother, wind, planet, earth, crib]
2                 [child, phrase, invent]
3               [great, gift, live, time]
4          [feel, silence, cardinal, sin]
Name: Sentences, dtype: object

In [258]:
dictionary = gensim.corpora.Dictionary(preprocess_sentences)
dictionary.filter_extremes(no_below=3)

corpus = [dictionary.doc2bow(doc) for doc in preprocess_sentences]

In [259]:
tfidf = models.TfidfModel(corpus)
corpus_tfidf = tfidf[corpus]

In [265]:
ida_model = gensim.models.LdaMulticore(corpus_tfidf, num_topics = 3, id2word=dictionary, passes=100)
for idx, topic in ida_model.print_topics(-1): 
    print('Topic: {} \nWords: {}\n'.format(idx, topic))

Topic: 0 
Words: 0.196*"life" + 0.110*"time" + 0.109*"play" + 0.093*"kid" + 0.092*"live" + 0.091*"write" + 0.089*"look" + 0.075*"end" + 0.012*"tell" + 0.012*"come"

Topic: 1 
Words: 0.164*"want" + 0.142*"child" + 0.135*"know" + 0.118*"like" + 0.091*"love" + 0.083*"come" + 0.077*"fuck" + 0.069*"tell" + 0.010*"look" + 0.010*"end"

Topic: 2 
Words: 0.327*"rap" + 0.144*"feel" + 0.114*"city" + 0.114*"science" + 0.114*"dream" + 0.012*"write" + 0.012*"kid" + 0.012*"child" + 0.012*"come" + 0.012*"tell"

